In [ ]:
#pip install psycopg2

In [6]:
# загрузка библиотек
import psycopg2
import pandas as pd

### 1. Подключение к базе данных

In [7]:
# загрузка библиотек
import psycopg2
import pandas as pd
# создание подключения к базе данных 'incident'
connect = psycopg2.connect(database='incident',
                       user='postgres',
                       password='postgres',
                       host='127.0.0.1',
                       port='5432')
print('База данных успешно открыта')
# создание объекта курсора подключения к базе данных 'incident'
cursor = connect.cursor()

База данных успешно открыта


In [8]:
# создание объекта курсора подключения к базе данных 'incident'
cursor = connect.cursor()

In [4]:
# вывод наименований таблиц в базе данных 'incident'
cursor.execute("SELECT table_name FROM information_schema.tables \
             WHERE table_schema NOT IN ('information_schema', 'pg_catalog') \
             AND table_schema IN('public', 'myschema');")
cursor.fetchall()

[('incidents',),
 ('houses',),
 ('adm_areas',),
 ('district_areas',),
 ('addresses',),
 ('work_types',),
 ('work_facts',)]

### 2. Создание таблиц

In [5]:
incidents = pd.read_csv("../data/incidents.csv")
incidents.head()

,UNOM,Количество обращений за последний месяц,Среднемесячное количество обращений
0,22830,414,0.316406
1,22830,414,0.316406
2,22830,414,0.316406
3,22830,414,0.316406
4,22830,414,0.316406


In [7]:
incidents.iloc[:,-1].min()

0.0028645833333333

In [6]:
incidents = incidents.iloc[:,[0, 2, 3]]
incidents.head()

,UNOM,Количество обращений за последний месяц,Среднемесячное количество обращений
0,22830,423,0.260901
1,22830,423,0.260901
2,22830,423,0.260901
3,22830,423,0.260901
4,22830,423,0.260901


In [13]:
incidents = incidents.drop_duplicates().reset_index(drop=True)
incidents.head()

,UNOM,Количество обращений за последний месяц,Среднемесячное количество обращений
0,22830,414,0.316406
1,27715,119,0.094792
2,8330,16,0.011979
3,4094,7,0.011458
4,20151,16,0.011458


In [14]:
incidents.columns

Index(['UNOM', 'Количество обращений за последний месяц',
       'Среднемесячное количество обращений'],
      dtype='object')

In [60]:
cursor.execute("""set datestyle to DMY;""")
connect.commit()

In [9]:
# удаление таблицы 'houses' если она существует  
cursor.execute("DROP TABLE IF EXISTS incidents;")
connect.commit()

In [62]:
# создание таблицы заказы - 'incidents' 
cursor.execute("""CREATE TABLE incidents( 
                id_incident INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, 
                unom INTEGER NOT NULL,
                data_created DATE NOT NULL,
                month_created INTEGER NOT NULL,
                calls_laste_month INTEGER NOT NULL,
                calls_aver_month NUMERIC(7,6) NOT NULL);""")
connect.commit() 

In [15]:
# создание таблицы заказы - 'incidents' 
cursor.execute("""CREATE TABLE incidents( 
                id_incident INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, 
                unom INTEGER NOT NULL,
                calls_laste_month INTEGER NOT NULL,
                calls_aver_month NUMERIC(7,6) NOT NULL);""")
connect.commit() 

In [16]:
incidents = incidents.values
incidents[:4]

array([[2.28300000e+04, 4.14000000e+02, 3.16406250e-01],
       [2.77150000e+04, 1.19000000e+02, 9.47916667e-02],
       [8.33000000e+03, 1.60000000e+01, 1.19791667e-02],
       [4.09400000e+03, 7.00000000e+00, 1.14583333e-02]])

In [17]:
# значение атрибутов в таблице 'incidents'
cursor.execute("SELECT column_name \
             FROM information_schema.columns \
             WHERE table_name = 'incidents';")
columnsname = [_[0] for _ in cursor.fetchall()]
print(columnsname)

['id_incident', 'unom', 'calls_laste_month', 'calls_aver_month']


In [21]:
# запись первой строки в таблицу - 'incidents' 
# cursor.executemany("""INSERT INTO incidents( 
#                       unom, data_created, month_created,
#                       calls_laste_month, calls_aver_month) 
#                       VALUES (%s, %s, %s, %s, %s);""", incidents)
# connect.commit() 

In [22]:
# запись первой строки в таблицу - 'incidents' 
cursor.executemany("""INSERT INTO incidents( 
                      unom, calls_laste_month, calls_aver_month) 
                      VALUES (%s, %s, %s);""", incidents)
connect.commit() 

In [9]:
# проверка записи данных в таблицу 'incidents'
cursor.execute("SELECT * FROM incidents LIMIT 10;")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[_.name for _ in cursor.description])
df

,id_incident,unom,calls_laste_month,calls_aver_month
0,1,22830,414,0.316406
1,2,27715,119,0.094792
2,3,8330,16,0.011979
3,4,4094,7,0.011458
4,5,20151,16,0.011458
5,6,10013,102,0.109896
6,7,21170,4,0.003125
7,8,3773,64,0.127083
8,9,27105,216,0.108333
9,10,27595,24,0.028125


In [24]:
# проверка записи данных в таблицу 'incidents'
cursor.execute("SELECT COUNT(*) FROM incidents;")
cursor.fetchone()

(371,)

In [25]:
len(incidents)

371

In [26]:
houses = pd.read_csv("../data/houses.csv")
houses.head()

,UNOM,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Статус МКД,Количество грузовых лифтов,Возраст дома
0,22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,49
1,22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,49
2,22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,49
3,22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,49
4,22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,49


In [27]:
houses.duplicated().sum()

43209

In [28]:
houses = houses.drop_duplicates().reset_index(drop=True)
houses.head()

,UNOM,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Статус МКД,Количество грузовых лифтов,Возраст дома
0,22830,I-515,9,12,21103,21088,15,панельные,12,0,в эксплуатации,0,49
1,27715,И-209А,14,1,4239,4239,0,из железобетонных сегментов,2,0,в эксплуатации,0,52
2,8330,П-46,12,2,4306,4266,40,панельные,4,0,в эксплуатации,0,37
3,4094,I-511-4/М22БН,5,4,3408,3160,248,кирпичные,0,0,в эксплуатации,0,61
4,20151,II-29-41/37,9,1,3140,3074,66,кирпичные,1,0,в эксплуатации,0,54


In [29]:
houses.iloc[:,[5, 6, 7]].agg(["max"])

,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен
max,40588,3711,панельные


In [30]:
houses.isna().sum()

UNOM                                   0
Серия проекта                          0
Количество этажей                      0
Количество подъездов                   0
Общая площадь                          0
Общая площадь жилых помещений          0
Общая площадь нежилых помещений        0
Материал стен                          0
Количество пассажирских лифтов         0
Количество грузопассажирских лифтов    0
Статус МКД                             0
Количество грузовых лифтов             0
Возраст дома                           0
dtype: int64

In [33]:
# удаление таблицы 'houses' если она существует  
cursor.execute("DROP TABLE IF EXISTS houses;")
connect.commit()

In [32]:
houses.columns

Index(['UNOM', 'Серия проекта', 'Количество этажей', 'Количество подъездов',
       'Общая площадь', 'Общая площадь жилых помещений',
       'Общая площадь нежилых помещений', 'Материал стен',
       'Количество пассажирских лифтов', 'Количество грузопассажирских лифтов',
       'Статус МКД', 'Количество грузовых лифтов', 'Возраст дома'],
      dtype='object')

In [34]:
# создание таблицы заказы - 'houses' 
cursor.execute("""CREATE TABLE houses( 
                id_house INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, 
                unom INTEGER UNIQUE NOT NULL,
                project_series TEXT NOT NULL,
                floor_value INTEGER NOT NULL,
                entrance_value INTEGER NOT NULL,
                total_area NUMERIC(6,1) NOT NULL,
                total_area_res NUMERIC(6,1) NOT NULL,
                total_area_non_res NUMERIC(6,1) NOT NULL,
                wall_material TEXT NOT NULL,
                num_elevator INTEGER NOT NULL,
                num_carco_elevator INTEGER NOT NULL,
                status_mcd TEXT NOT NULL,
                num_freight_elevastor INTEGER NOT NULL,
                year INTEGER NOT NULL);""")
connect.commit() 

In [35]:
# значение атрибутов в таблице 'houses'
cursor.execute("SELECT column_name \
             FROM information_schema.columns \
             WHERE table_name = 'houses';")
columnsname = [_[0] for _ in cursor.fetchall()]
print(columnsname)

['id_house', 'unom', 'project_series', 'floor_value', 'entrance_value', 'total_area', 'total_area_res', 'total_area_non_res', 'wall_material', 'num_elevator', 'num_carco_elevator', 'status_mcd', 'num_freight_elevastor', 'year']


In [36]:
houses = houses.values
houses[:2]

array([[22830, 'I-515', 9, 12, 21103, 21088, 15, 'панельные', 12, 0,
        'в эксплуатации', 0, 49],
       [27715, 'И-209А', 14, 1, 4239, 4239, 0,
        'из железобетонных сегментов', 2, 0, 'в эксплуатации', 0, 52]],
      dtype=object)

In [37]:
# запись первой строки в таблицу - 'houses' 
cursor.executemany("""INSERT INTO houses( 
                      unom, project_series, floor_value, entrance_value, total_area,
                      total_area_res, total_area_non_res, wall_material, num_elevator,
                      num_carco_elevator, status_mcd, num_freight_elevastor, year) 
                      VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""", houses)
connect.commit() 

In [38]:
# проверка записи данных в таблицу 'houses'
cursor.execute("SELECT * FROM houses LIMIT 10;")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[_.name for _ in cursor.description])
df

,id_house,unom,project_series,floor_value,entrance_value,total_area,total_area_res,total_area_non_res,wall_material,num_elevator,num_carco_elevator,status_mcd,num_freight_elevastor,year
0,1,22830,I-515,9,12,21103.0,21088.0,15.0,панельные,12,0,в эксплуатации,0,49
1,2,27715,И-209А,14,1,4239.0,4239.0,0.0,из железобетонных сегментов,2,0,в эксплуатации,0,52
2,3,8330,П-46,12,2,4306.0,4266.0,40.0,панельные,4,0,в эксплуатации,0,37
3,4,4094,I-511-4/М22БН,5,4,3408.0,3160.0,248.0,кирпичные,0,0,в эксплуатации,0,61
4,5,20151,II-29-41/37,9,1,3140.0,3074.0,66.0,кирпичные,1,0,в эксплуатации,0,54
5,6,10013,П-49 Д,9,8,14312.0,14312.0,0.0,панельные,8,0,в эксплуатации,0,51
6,7,21170,нет серии проекта,5,4,3451.0,3138.0,313.0,кирпичные,0,0,в эксплуатации,0,62
7,8,3773,П-49 Д,9,6,10527.0,10527.0,0.0,панельные,6,0,в эксплуатации,0,50
8,9,27105,П-18/22,12,1,3624.0,3624.0,0.0,из железобетонных сегментов,2,0,в эксплуатации,0,50
9,10,27595,нет серии проекта,5,3,3396.0,2043.0,1353.0,кирпичные,0,0,в эксплуатации,0,64


In [42]:
df.nunique()

id_house                 10
unom                     10
project_series            8
floor_value               4
entrance_value            7
total_area               10
total_area_res           10
total_area_non_res        7
wall_material             3
num_elevator              7
num_carco_elevator        1
status_mcd                1
num_freight_elevastor     1
year                      9
dtype: int64

In [39]:
# проверка записи данных в таблицу 'houses'
cursor.execute("SELECT COUNT(*) FROM houses;")
cursor.fetchone()

(371,)

In [40]:
len(houses)

371

In [30]:
cursor.execute("SELECT distinct(project_series) FROM houses;")
cursor.fetchall()

[('П-46',),
 ('II-49-04/Ю',),
 ('10-03-711',),
 ('I-511',),
 ('П-43',),
 ('I-510',),
 ('П-3/17',),
 ('II-68-01',),
 ('IМГ-601Д',),
 ('П-3/16',),
 ('II-49-06',),
 ('нет данных',),
 ('КОПЭ',),
 ('П-57',),
 ('ПД-4/12',),
 ('I-515',),
 ('II-49-06-ЮД',),
 ('КТЖС',),
 ('II-29-05/Ю37',),
 ('II-29-04/Ю37',),
 ('II-29-41/37',),
 ('II-68-02/16М',),
 ('И-209А',),
 ('П-55',),
 ('П-18/22',),
 ('П-29-03/Ю37',),
 ('П-44',),
 ('П-46м',),
 ('П-30',),
 ('II-18-21/12',),
 ('индивидуальный проект',),
 ('I-511-4/М22БН',),
 ('II-18-01/12',),
 ('П-31',),
 ('П-3/22',),
 ('П-68',),
 ('И-700А',),
 ('П-46-2/12в',),
 ('И-522',),
 ('П-29',),
 ('П-49 Д',),
 ('П-47',)]

In [78]:
df.columns

Index(['unom', 'id_incident', 'data_created', 'month_created',
       'calls_laste_month', 'calls_aver_month', 'id_house', 'year',
       'project_series', 'floor_value', 'entrance_value', 'total_area',
       'total_area_res', 'total_area_non_res', 'wall_material', 'num_elevator',
       'num_carco_elevator', 'status_mcd', 'num_freight_elevastor'],
      dtype='object')

In [32]:
# проверка записи данных в таблицу 'work_facts'
# cursor.execute("""SELECT unom, data_created, month_created, calls_laste_month,
#                   calls_aver_month, year, project_series, floor_value,
#                   entrance_value, total_area, total_area_res, total_area_non_res,
#                   wall_material, num_elevator, num_carco_elevator, status_mcd,
#                   num_freight_elevastor
#                   FROM incidents
#                   INNER JOIN houses USING(unom);""")
# rows = cursor.fetchall()
# df = pd.DataFrame(rows, columns=[_.name for _ in cursor.description])
# df

In [37]:
# проверка записи данных в таблицу 'work_facts'
cursor.execute("""SELECT unom, calls_laste_month,
                  calls_aver_month, year, project_series, floor_value,
                  entrance_value, total_area, total_area_res, total_area_non_res,
                  wall_material, num_elevator, num_carco_elevator, status_mcd,
                  num_freight_elevastor
                  FROM incidents
                  INNER JOIN houses USING(unom);""")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[_.name for _ in cursor.description])
df

,unom,calls_laste_month,calls_aver_month,year,project_series,floor_value,entrance_value,total_area,total_area_res,total_area_non_res,wall_material,num_elevator,num_carco_elevator,status_mcd,num_freight_elevastor
0,22830,423,0.260901,1973,I-515,9,12,21103.0,21088.0,15.0,панельные,12,0,в эксплуатации,0
1,22830,423,0.260901,1973,I-515,9,12,21103.0,21088.0,15.0,панельные,12,0,в эксплуатации,0
2,22830,423,0.260901,1973,I-515,9,12,21103.0,21088.0,15.0,панельные,12,0,в эксплуатации,0
3,22830,423,0.260901,1973,I-515,9,12,21103.0,21088.0,15.0,панельные,12,0,в эксплуатации,0
4,22830,423,0.260901,1973,I-515,9,12,21103.0,21088.0,15.0,панельные,12,0,в эксплуатации,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46425,28393,126,0.067783,1975,И-209А,14,1,4263.0,4263.0,0.0,каменные и бетонные,2,0,в эксплуатации,0
46426,28393,126,0.067783,1975,И-209А,14,1,4263.0,4263.0,0.0,каменные и бетонные,2,0,в эксплуатации,0
46427,28393,126,0.067783,1975,И-209А,14,1,4263.0,4263.0,0.0,каменные и бетонные,2,0,в эксплуатации,0
46428,28393,126,0.067783,1975,И-209А,14,1,4263.0,4263.0,0.0,каменные и бетонные,2,0,в эксплуатации,0


In [77]:
df.shape

(371, 19)

In [4]:
addresses = pd.read_csv("../data/addresses.csv", usecols=["unom", "ADM_AREA", "DISTRICT", "SIMPLE_ADDRESS", "LATITUDE", "LONGITUDE"])
addresses.head()

,unom,ADM_AREA,DISTRICT,SIMPLE_ADDRESS,LATITUDE,LONGITUDE
0,2115191,Восточный,Соколиная Гора,"Малая Семёновская улица, дом 13",55.785515,37.712773
1,2404461,Восточный,Косино-Ухтомский,"улица Дмитриевского, дом 5",55.709856,37.884794
2,2501561,Восточный,Соколиная Гора,"Вольная улица, дом 30, строение 10",55.776361,37.738259
3,8027,Восточный,Восточный,"Западная улица, дом 10",55.807917,37.863290
4,2105301,Восточный,Соколиная Гора,"9-я улица Соколиной Горы, дом 17А",55.773403,37.735270


In [9]:
addresses[addresses["SIMPLE_ADDRESS"].apply(lambda x: "Новокосинская улица, дом 51" in x)]

,unom,ADM_AREA,DISTRICT,SIMPLE_ADDRESS,LATITUDE,LONGITUDE
2312,16567,Восточный,Новокосино,"Новокосинская улица, дом 51, корпус 3",55.745327,37.878507
4186,16565,Восточный,Новокосино,"Новокосинская улица, дом 51, корпус 1",55.744529,37.877864
5204,16566,Восточный,Новокосино,"Новокосинская улица, дом 51, корпус 2",55.744833,37.878457


In [1]:
# загрузка библиотек
import psycopg2

In [2]:
# создание подключения к базе данных 'incident'
connect = psycopg2.connect(database='incident',
                       user='postgres',
                       password='postgres',
                       host='127.0.0.1',
                       port='5432')
print('База данных успешно открыта')

База данных успешно открыта


In [3]:
# создание объекта курсора подключения к базе данных 'incident'
cursor = connect.cursor()

In [4]:
# адрес дома
addresse_title = "Новокосинская улица, дом 51, корпус 2"
addresse_title

'Новокосинская улица, дом 51, корпус 2'

In [5]:
# получение unom по адресу дома (addresse_title)
cursor.execute("""SELECT unom
                  FROM addresses
                  WHERE addresse_title = %s;""",
                  (addresse_title,))
unom = cursor.fetchone()[0]
unom

16566

In [39]:
addresses.shape

(6762, 6)

In [40]:
addresses.nunique()

unom              6762
ADM_AREA             1
DISTRICT            16
SIMPLE_ADDRESS    6758
LATITUDE          6759
LONGITUDE         6760
dtype: int64

#### 2.1. Таблица заказы - 'adm_area'

In [41]:
# удаление таблицы 'adm_areas' если она существует  
cursor.execute("DROP TABLE IF EXISTS adm_areas;")
connect.commit()

In [42]:
# создание таблицы заказы - 'adm_areas' 
cursor.execute("""CREATE TABLE adm_areas( 
                id_adm_area INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, 
                adm_area_name TEXT UNIQUE NOT NULL);""")
connect.commit() 

In [43]:
# значение атрибутов в таблице 'adm_areas'
cursor.execute("SELECT column_name \
             FROM information_schema.columns \
             WHERE table_name = 'adm_areas';")
columnsname = [_[0] for _ in cursor.fetchall()]
print(columnsname)

['id_adm_area', 'adm_area_name']


In [44]:
adm_area_name = addresses["ADM_AREA"].unique()
adm_area_name

array(['Восточный'], dtype=object)

In [45]:
# запись первой строки в таблицу - 'adm_areas' 
cursor.execute("""INSERT INTO adm_areas( 
                adm_area_name) 
                VALUES (%s);""", adm_area_name)
connect.commit() 

In [46]:
# проверка записи данных в таблицу 'adm_areas'
cursor.execute("SELECT * FROM adm_areas LIMIT 10;")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[_.name for _ in cursor.description])
df

,id_adm_area,adm_area_name
0,1,Восточный


#### 2.2. Таблица заказы - 'district areas'

In [47]:
# удаление таблицы 'district_areas' если она существует  
cursor.execute("DROP TABLE IF EXISTS district_areas;")
connect.commit()

In [48]:
# создание таблицы заказы - 'district_areas' 
cursor.execute("""CREATE TABLE district_areas( 
                id_district_area INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, 
                district_area_name TEXT UNIQUE NOT NULL);""")
connect.commit() 

In [49]:
# значение атрибутов в таблице 'district_areas'
cursor.execute("SELECT column_name \
             FROM information_schema.columns \
             WHERE table_name = 'district_areas';")
columnsname = [_[0] for _ in cursor.fetchall()]
print(columnsname)

['id_district_area', 'district_area_name']


In [50]:
district_names = addresses[["DISTRICT"]].drop_duplicates().values
district_names[:7]

array([['Соколиная Гора'],
       ['Косино-Ухтомский'],
       ['Восточный'],
       ['Измайлово'],
       ['Гольяново'],
       ['Богородское'],
       ['Перово']], dtype=object)

In [51]:
# запись первой строки в таблицу - 'district_areas' 
cursor.executemany("""INSERT INTO district_areas( 
                      district_area_name) 
                      VALUES (%s);""", district_names)
connect.commit() 

In [52]:
# проверка записи данных в таблицу 'district_areas'
cursor.execute("SELECT * FROM district_areas LIMIT 5;")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[_.name for _ in cursor.description])
df

,id_district_area,district_area_name
0,1,Соколиная Гора
1,2,Косино-Ухтомский
2,3,Восточный
3,4,Измайлово
4,5,Гольяново


In [53]:
# проверка записи данных в таблицу 'district_areas'
cursor.execute("SELECT COUNT(*) FROM district_areas;")
cursor.fetchone()

(16,)

In [54]:
len(district_names)

16

In [55]:
addresses.head()

,unom,ADM_AREA,DISTRICT,SIMPLE_ADDRESS,LATITUDE,LONGITUDE
0,2115191,Восточный,Соколиная Гора,"Малая Семёновская улица, дом 13",55.785515,37.712773
1,2404461,Восточный,Косино-Ухтомский,"улица Дмитриевского, дом 5",55.709856,37.884794
2,2501561,Восточный,Соколиная Гора,"Вольная улица, дом 30, строение 10",55.776361,37.738259
3,8027,Восточный,Восточный,"Западная улица, дом 10",55.807917,37.863290
4,2105301,Восточный,Соколиная Гора,"9-я улица Соколиной Горы, дом 17А",55.773403,37.735270


#### 2.3. Таблица заказы - 'addresses'

In [56]:
# удаление таблицы 'addresses' если она существует  
cursor.execute("DROP TABLE IF EXISTS addresses;")
connect.commit()

In [57]:
# создание таблицы заказы - 'addresses' 
cursor.execute("""CREATE TABLE addresses( 
                id_addresse INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY,
                unom INTEGER NOT NULL,
                id_adm_area INTEGER NOT NULL,
                id_district_area INTEGER NOT NULL,
                addresse_title TEXT NOT NULL,
                latitude NUMERIC(8,6) NOT NULL,
                longitude NUMERIC(8,6) NOT NULL);""")
connect.commit() 

In [58]:
def get_id_district_area(district_area_name):
    
    cursor.execute("""SELECT id_district_area
                      FROM district_areas
                      WHERE district_area_name = %s;""", (district_area_name,))
    id_district_area = cursor.fetchone()[0]

    return id_district_area

In [59]:
get_id_district_area("Соколиная Гора")

1

In [60]:
def get_id_adm_area(adm_area_name):
    
    cursor.execute("""SELECT id_adm_area
                      FROM adm_areas
                      WHERE adm_area_name = %s;""", (adm_area_name,))
    id_adm_area = cursor.fetchone()[0]

    return id_adm_area

In [61]:
get_id_adm_area("Восточный")

1

In [62]:
# значение атрибутов в таблице 'addresses'
cursor.execute("SELECT column_name \
             FROM information_schema.columns \
             WHERE table_name = 'addresses';")
columnsname = [_[0] for _ in cursor.fetchall()]
print(columnsname)

['id_addresse', 'unom', 'id_adm_area', 'id_district_area', 'addresse_title', 'latitude', 'longitude']


In [63]:
def insert_addresse(unom, id_adm_area, id_district_area, addresse_title, latitude, longitude):
    # запись первой строки в таблицу - 'addresses' 
    cursor.execute("""INSERT INTO addresses( 
                      unom, id_adm_area, id_district_area, addresse_title, latitude, longitude) 
                      VALUES (%s, %s, %s, %s, %s, %s);""",
                      (unom, id_adm_area, id_district_area, addresse_title, latitude, longitude))
    connect.commit() 

In [64]:
for i in range(len(addresses)):

    unom, adm_area_name, district_area_name, addresse_title, latitude, longitude = addresses.loc[i].values

    id_adm_area = get_id_adm_area(adm_area_name)
    id_district_area = get_id_district_area(district_area_name)
    insert_addresse(int(unom), id_adm_area, id_district_area, addresse_title, float(latitude), float(longitude))

In [65]:
# проверка записи данных в таблицу 'addresses'
cursor.execute("SELECT * FROM addresses LIMIT 5;")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[_.name for _ in cursor.description])
df

,id_addresse,unom,id_adm_area,id_district_area,addresse_title,latitude,longitude
0,1,2115191,1,1,"Малая Семёновская улица, дом 13",55.785515,37.712773
1,2,2404461,1,2,"улица Дмитриевского, дом 5",55.709856,37.884794
2,3,2501561,1,1,"Вольная улица, дом 30, строение 10",55.776361,37.738259
3,4,8027,1,3,"Западная улица, дом 10",55.807917,37.863290
4,5,2105301,1,1,"9-я улица Соколиной Горы, дом 17А",55.773403,37.735270


In [66]:
# проверка записи данных в таблицу 'addresses'
cursor.execute("SELECT COUNT(*) FROM addresses;")
cursor.fetchone()

(6762,)

In [67]:
len(addresses)

6762

In [68]:
# проверка записи данных в таблицу 'addresses'
cursor.execute("""SELECT unom, adm_area_name, district_area_name, addresse_title, latitude, longitude
                  FROM addresses
                  INNER JOIN adm_areas USING(id_adm_area)
                  INNER JOIN district_areas USING(id_district_area)
                  LIMIT 5;""")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[_.name for _ in cursor.description])
df

,unom,adm_area_name,district_area_name,addresse_title,latitude,longitude
0,2115191,Восточный,Соколиная Гора,"Малая Семёновская улица, дом 13",55.785515,37.712773
1,2404461,Восточный,Косино-Ухтомский,"улица Дмитриевского, дом 5",55.709856,37.884794
2,2501561,Восточный,Соколиная Гора,"Вольная улица, дом 30, строение 10",55.776361,37.738259
3,8027,Восточный,Восточный,"Западная улица, дом 10",55.807917,37.863290
4,2105301,Восточный,Соколиная Гора,"9-я улица Соколиной Горы, дом 17А",55.773403,37.735270


In [69]:
# проверка записи данных в таблицу 'addresses'
cursor.execute("""SELECT unom, latitude, longitude
                  FROM addresses
                  WHERE unom in %s;""", ((2115191, 2404461),))
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[_.name for _ in cursor.description])
df

,unom,latitude,longitude
0,2115191,55.785515,37.712773
1,2404461,55.709856,37.884794


In [71]:
# загрузка данных виды работ по капитальному ремонту многоквартирных домов (справочник видов капитальных работ)
type_work_laste_df = pd.read_csv("../data/type_work_laste_df.csv")
type_work_laste_df.head()

,Код,Наименование,Наименование объекта общего имущества,Тип работ,Группа работ,Сокращенное наименование работы,Виды работ в 2022 году
0,105,строительный контроль,Строительный контроль,Услуга,Стройконтроль,СК,работы не проводились
1,26,"ремонт подъездов, направленный на восстановлен...",Подъезд,Работа,СМР,Подъезд,"ремонт подъездов, направленный на восстановлен..."
2,9,"ремонт фасада (замена оконных блоков, располож...",Фасад(оконные блоки),Работа,СМР,Окна,"замена оконных блоков, расположенных в помещен..."
3,104,проведение экспертизы проектной документации п...,проведение экспертизы,Услуга,СМР,проведение экспертизы,работы не проводились
4,20,разработка проектной документации,разработка проектной документации,Услуга,СМР,ПСД,работы не проводились


In [72]:
# загрузка данных работы по капитальному ремонту, проведенные в многоквартирных домах
overhaul_df = pd.read_csv("../data/overhaul_df.csv")
overhaul_df.head()

,global_id,PERIOD,WORK_NAME,NUM_ENTRANCE,ElevatorNumber,PLAN_DATE_START,PLAN_DATE_END,FACT_DATE_START,FACT_DATE_END,AdmArea,District,Address,UNOM
0,351060,2022,замена лифтового оборудования,1,1,15.07.2022,28.08.2022,27.06.2022,14.10.2022,Восточный административный округ,район Богородское,"Российская Федерация, город Москва, внутригоро...",20151
1,430496,2022,замена лифтового оборудования,1,1,30.09.2022,14.10.2022,30.09.2022,21.10.2022,Восточный административный округ,район Богородское,"Российская Федерация, город Москва, внутригоро...",20151
2,348580,2022,ремонт внутридомовых инженерных систем водоотв...,1,нет лифта,21.02.2022,20.06.2022,01.02.2022,25.05.2022,Восточный административный округ,район Богородское,"Российская Федерация, город Москва, внутригоро...",11466
3,348581,2022,ремонт внутридомовых инженерных систем теплосн...,1,нет лифта,21.04.2022,10.07.2022,07.05.2022,03.08.2022,Восточный административный округ,район Богородское,"Российская Федерация, город Москва, внутригоро...",11466
4,348578,2022,ремонт внутридомовых инженерных систем холодно...,1,нет лифта,21.02.2022,20.06.2022,01.02.2022,27.06.2022,Восточный административный округ,район Богородское,"Российская Федерация, город Москва, внутригоро...",11466


In [73]:
overhaul_df = overhaul_df.iloc[:,[-1, 1, 2, 3, 4, 5, 6, 7, 8]].merge(type_work_laste_df, left_on="WORK_NAME", right_on="Виды работ в 2022 году").drop(columns=["Виды работ в 2022 году"])
overhaul_df.head()

,UNOM,PERIOD,WORK_NAME,NUM_ENTRANCE,ElevatorNumber,PLAN_DATE_START,PLAN_DATE_END,FACT_DATE_START,FACT_DATE_END,Код,Наименование,Наименование объекта общего имущества,Тип работ,Группа работ,Сокращенное наименование работы
0,20151,2022,замена лифтового оборудования,1,1,15.07.2022,28.08.2022,27.06.2022,14.10.2022,6,замена лифтового оборудования,Лифт,Работа,Лифты,Лифт
1,20151,2022,замена лифтового оборудования,1,1,30.09.2022,14.10.2022,30.09.2022,21.10.2022,6,замена лифтового оборудования,Лифт,Работа,Лифты,Лифт
2,8576,2022,замена лифтового оборудования,3,3,15.07.2022,28.08.2022,27.06.2022,06.10.2022,6,замена лифтового оборудования,Лифт,Работа,Лифты,Лифт
3,21908,2022,замена лифтового оборудования,1,1,23.06.2022,11.08.2022,23.06.2022,31.10.2022,6,замена лифтового оборудования,Лифт,Работа,Лифты,Лифт
4,21910,2022,замена лифтового оборудования,1,1,23.06.2022,11.08.2022,23.06.2022,31.10.2022,6,замена лифтового оборудования,Лифт,Работа,Лифты,Лифт


In [74]:
overhaul_df.isna().sum()

UNOM                                     0
PERIOD                                   0
WORK_NAME                                0
NUM_ENTRANCE                             0
ElevatorNumber                           0
PLAN_DATE_START                          0
PLAN_DATE_END                            0
FACT_DATE_START                          0
FACT_DATE_END                            0
Код                                      0
Наименование                             0
Наименование объекта общего имущества    0
Тип работ                                0
Группа работ                             0
Сокращенное наименование работы          0
dtype: int64

In [75]:
# удаление таблицы 'work_types' если она существует  
cursor.execute("DROP TABLE IF EXISTS work_types;")
connect.commit()

In [76]:
# создание таблицы заказы - 'work_types' 
cursor.execute("""CREATE TABLE work_types( 
                id_work INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY,
                work_name_example TEXT NOT NULL,
                work_code INTEGER NOT NULL,
                work_name TEXT NOT NULL,
                name_property TEXT NOT NULL,
                work_type TEXT NOT NULL,
                work_group TEXT NOT NULL,
                work_shot_name TEXT NOT NULL);""")
connect.commit() 

In [77]:
work_types = overhaul_df.iloc[:,[2, -6, -5, -4, -3, -2, -1]].drop_duplicates().values
work_types[:3]

array([['замена лифтового оборудования', 6,
        'замена лифтового оборудования', 'Лифт', 'Работа', 'Лифты',
        'Лифт'],
       ['ремонт внутридомовых инженерных систем водоотведения (канализации) (выпуски и сборные трубопроводы)',
        5,
        'ремонт внутридомовых инженерных систем водоотведения (канализации) (выпуски и сборные трубопроводы)',
        'Внутридомовые системы канализации (магистрали)', 'Работа',
        'СМР', 'КАН-М'],
       ['ремонт внутридомовых инженерных систем теплоснабжения (разводящие магистрали)',
        3,
        'ремонт внутридомовых инженерных систем теплоснабжения (разводящие магистрали)',
        'Внутридомовые системы отопления (магистрали)', 'Работа', 'СМР',
        'ЦО-М']], dtype=object)

In [78]:
# запись первой строки в таблицу - 'work_types' 
cursor.executemany("""INSERT INTO work_types( 
                      work_name_example, work_code, work_name,
                      name_property, work_type, work_group,
                      work_shot_name) 
                      VALUES (%s, %s, %s, %s, %s, %s, %s);""", work_types)
connect.commit() 

In [79]:
# проверка записи данных в таблицу 'work_types'
cursor.execute("""SELECT *
                  FROM work_types
                  LIMIT 5;""")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[_.name for _ in cursor.description])
df

,id_work,work_name_example,work_code,work_name,name_property,work_type,work_group,work_shot_name
0,1,замена лифтового оборудования,6,замена лифтового оборудования,Лифт,Работа,Лифты,Лифт
1,2,ремонт внутридомовых инженерных систем водоотв...,5,ремонт внутридомовых инженерных систем водоотв...,Внутридомовые системы канализации (магистрали),Работа,СМР,КАН-М
2,3,ремонт внутридомовых инженерных систем теплосн...,3,ремонт внутридомовых инженерных систем теплосн...,Внутридомовые системы отопления (магистрали),Работа,СМР,ЦО-М
3,4,ремонт внутридомовых инженерных систем холодно...,4,ремонт внутридомовых инженерных систем холодно...,Внутридомовые системы холодного водоснабжения ...,Работа,СМР,ХВС-М
4,5,ремонт внутридомовых инженерных систем горячег...,4,ремонт внутридомовых инженерных систем горячег...,Внутридомовые инженерные системы горячего водо...,Работа,СМР,ГВС-М


In [80]:
# проверка записи данных в таблицу 'work_types'
cursor.execute("""SELECT COUNT(*)
                  FROM work_types;""")
cursor.fetchone()

(19,)

In [81]:
len(work_types)

19

In [82]:
overhaul_df = overhaul_df.iloc[:,[0, 2, 1, 3, 4, 5, 6, 7, 8]]
overhaul_df.head()

,UNOM,WORK_NAME,PERIOD,NUM_ENTRANCE,ElevatorNumber,PLAN_DATE_START,PLAN_DATE_END,FACT_DATE_START,FACT_DATE_END
0,20151,замена лифтового оборудования,2022,1,1,15.07.2022,28.08.2022,27.06.2022,14.10.2022
1,20151,замена лифтового оборудования,2022,1,1,30.09.2022,14.10.2022,30.09.2022,21.10.2022
2,8576,замена лифтового оборудования,2022,3,3,15.07.2022,28.08.2022,27.06.2022,06.10.2022
3,21908,замена лифтового оборудования,2022,1,1,23.06.2022,11.08.2022,23.06.2022,31.10.2022
4,21910,замена лифтового оборудования,2022,1,1,23.06.2022,11.08.2022,23.06.2022,31.10.2022


In [83]:
overhaul_df["WORK_NAME"].value_counts()

WORK_NAME
замена лифтового оборудования                                                                                                                                                       285
ремонт подвальных помещений, относящихся к общему имуществу в многоквартирном доме                                                                                                  188
ремонт внутридомовых инженерных систем теплоснабжения (разводящие магистрали)                                                                                                       175
ремонт внутридомовых инженерных систем холодного водоснабжения (разводящие магистрали)                                                                                              161
ремонт фасадов                                                                                                                                                                      156
ремонт внутридомовых инженерных систем водоотведения (канализации) (вы

In [84]:
overhaul_df.groupby(["UNOM", "PLAN_DATE_START"], as_index=False)["WORK_NAME"].nunique().sort_values(by="WORK_NAME")

,UNOM,PLAN_DATE_START,WORK_NAME
0,365,01.04.2022,1
646,24227,21.05.2022,1
647,24237,01.04.2022,1
648,24237,05.04.2022,1
650,24237,11.05.2022,1
...,...,...,...
656,24239,21.03.2022,6
394,18508,01.06.2022,6
177,8463,21.07.2022,6
1,365,11.03.2022,6


In [85]:
overhaul_df.head()

,UNOM,WORK_NAME,PERIOD,NUM_ENTRANCE,ElevatorNumber,PLAN_DATE_START,PLAN_DATE_END,FACT_DATE_START,FACT_DATE_END
0,20151,замена лифтового оборудования,2022,1,1,15.07.2022,28.08.2022,27.06.2022,14.10.2022
1,20151,замена лифтового оборудования,2022,1,1,30.09.2022,14.10.2022,30.09.2022,21.10.2022
2,8576,замена лифтового оборудования,2022,3,3,15.07.2022,28.08.2022,27.06.2022,06.10.2022
3,21908,замена лифтового оборудования,2022,1,1,23.06.2022,11.08.2022,23.06.2022,31.10.2022
4,21910,замена лифтового оборудования,2022,1,1,23.06.2022,11.08.2022,23.06.2022,31.10.2022


In [86]:
cursor.execute("""set datestyle to DMY;""")
connect.commit()

In [87]:
# удаление таблицы 'work_facts' если она существует  
cursor.execute("DROP TABLE IF EXISTS work_facts;")
connect.commit()

In [88]:
# создание таблицы заказы - 'work_facts' 
cursor.execute("""CREATE TABLE work_facts( 
                id_work_fact INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY,
                unom INTEGER NOT NULL,
                id_work INTEGER NOT NULL,
                period INTEGER NOT NULL,
                num_entrance INTEGER NOT NULL,
                elevator_num TEXT NOT NULL,
                plan_date_start DATE NOT NULL,
                plan_date_end DATE NOT NULL,
                fact_date_start DATE,
                fact_date_end DATE);""")
connect.commit() 

In [89]:
def get_id_work(work_name_example):
    
    cursor.execute("""SELECT id_work
                      FROM work_types
                      WHERE work_name_example = %s;""", (work_name_example,))
    id_work = cursor.fetchone()[0]

    return id_work

In [90]:
get_id_work("замена лифтового оборудования")

1

In [91]:
overhaul_df.head()

,UNOM,WORK_NAME,PERIOD,NUM_ENTRANCE,ElevatorNumber,PLAN_DATE_START,PLAN_DATE_END,FACT_DATE_START,FACT_DATE_END
0,20151,замена лифтового оборудования,2022,1,1,15.07.2022,28.08.2022,27.06.2022,14.10.2022
1,20151,замена лифтового оборудования,2022,1,1,30.09.2022,14.10.2022,30.09.2022,21.10.2022
2,8576,замена лифтового оборудования,2022,3,3,15.07.2022,28.08.2022,27.06.2022,06.10.2022
3,21908,замена лифтового оборудования,2022,1,1,23.06.2022,11.08.2022,23.06.2022,31.10.2022
4,21910,замена лифтового оборудования,2022,1,1,23.06.2022,11.08.2022,23.06.2022,31.10.2022


In [92]:
def insert_work_facts(unom, id_work, period, num_entrance, elevator_num, plan_date_start, plan_date_end, fact_date_start, fact_date_end):
    # запись первой строки в таблицу - 'addresses' 
    cursor.execute("""INSERT INTO work_facts( 
                        unom, id_work, period, num_entrance, elevator_num, plan_date_start, plan_date_end, fact_date_start, fact_date_end) 
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);""",
                        (unom, id_work, period, num_entrance, elevator_num,
                         plan_date_start, plan_date_end, fact_date_start, fact_date_end))
    connect.commit() 

In [93]:
for i in range(len(overhaul_df)):

    unom, work_name_example, period, num_entrance, elevator_num, plan_date_start, plan_date_end, fact_date_start, fact_date_end = overhaul_df.loc[i].values

    id_work = get_id_work(work_name_example)
    insert_work_facts(int(unom), id_work, int(period), int(num_entrance), elevator_num, plan_date_start, plan_date_end, fact_date_start, fact_date_end)

In [94]:
# проверка записи данных в таблицу 'work_facts'
cursor.execute("""SELECT *
                  FROM work_facts
                  LIMIT 5;""")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[_.name for _ in cursor.description])
df

,id_work_fact,unom,id_work,period,num_entrance,elevator_num,plan_date_start,plan_date_end,fact_date_start,fact_date_end
0,1,20151,1,2022,1,1,2022-07-15,2022-08-28,2022-06-27,2022-10-14
1,2,20151,1,2022,1,1,2022-09-30,2022-10-14,2022-09-30,2022-10-21
2,3,8576,1,2022,3,3,2022-07-15,2022-08-28,2022-06-27,2022-10-06
3,4,21908,1,2022,1,1,2022-06-23,2022-08-11,2022-06-23,2022-10-31
4,5,21910,1,2022,1,1,2022-06-23,2022-08-11,2022-06-23,2022-10-31


In [95]:
# проверка записи данных в таблицу 'work_facts'
cursor.execute("""SELECT COUNT(*)
                  FROM work_facts;""")
cursor.fetchone()

(1747,)

In [96]:
len(overhaul_df)

1747

In [97]:
# проверка записи данных в таблицу 'work_facts'
cursor.execute("""SELECT unom, adm_area_name, district_area_name, addresse_title, work_name_example, period, num_entrance, elevator_num, plan_date_start, plan_date_end, latitude, longitude
                  FROM work_facts
                  INNER JOIN work_types USING(id_work)
                  INNER JOIN addresses USING(unom)
                  INNER JOIN adm_areas USING(id_adm_area)
                  INNER JOIN district_areas USING(id_district_area);""")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[_.name for _ in cursor.description])
df

,unom,adm_area_name,district_area_name,addresse_title,work_name_example,period,num_entrance,elevator_num,plan_date_start,plan_date_end,latitude,longitude
0,20151,Восточный,Богородское,"Ивантеевская улица, дом 1, корпус 6",замена лифтового оборудования,2022,1,1,2022-07-15,2022-08-28,55.823447,37.717055
1,20151,Восточный,Богородское,"Ивантеевская улица, дом 1, корпус 6",замена лифтового оборудования,2022,1,1,2022-09-30,2022-10-14,55.823447,37.717055
2,8576,Восточный,Богородское,"Игральная улица, дом 10",замена лифтового оборудования,2022,3,3,2022-07-15,2022-08-28,55.815624,37.714302
3,21908,Восточный,Богородское,"бульвар Маршала Рокоссовского, дом 8, корпус 1",замена лифтового оборудования,2022,1,1,2022-06-23,2022-08-11,55.811977,37.727342
4,21910,Восточный,Богородское,"бульвар Маршала Рокоссовского, дом 8, корпус 3",замена лифтового оборудования,2022,1,1,2022-06-23,2022-08-11,55.811678,37.727592
...,...,...,...,...,...,...,...,...,...,...,...,...
1742,12178,Восточный,Ивановское,"Большой Купавенский проезд, дом 10",ремонт внутреннего водостока,2022,1,нет лифта,2022-11-11,2022-12-31,55.777302,37.820529
1743,12174,Восточный,Ивановское,"Большой Купавенский проезд, дом 2",ремонт внутреннего водостока,2022,1,нет лифта,2022-11-21,2022-12-31,55.773704,37.820432
1744,14953,Восточный,Вешняки,"улица Молдагуловой, дом 12, корпус 2",ремонт внутридомовых инженерных систем водоотв...,2022,1,нет лифта,2022-04-21,2022-08-20,55.725410,37.813979
1745,18508,Восточный,Измайлово,"3-я Парковая улица, дом 7/17",ремонт внутридомовых инженерных систем водоотв...,2022,1,нет лифта,2022-06-01,2022-09-30,55.790469,37.783655


In [98]:
df.head()

,unom,adm_area_name,district_area_name,addresse_title,work_name_example,period,num_entrance,elevator_num,plan_date_start,plan_date_end,latitude,longitude
0,20151,Восточный,Богородское,"Ивантеевская улица, дом 1, корпус 6",замена лифтового оборудования,2022,1,1,2022-07-15,2022-08-28,55.823447,37.717055
1,20151,Восточный,Богородское,"Ивантеевская улица, дом 1, корпус 6",замена лифтового оборудования,2022,1,1,2022-09-30,2022-10-14,55.823447,37.717055
2,8576,Восточный,Богородское,"Игральная улица, дом 10",замена лифтового оборудования,2022,3,3,2022-07-15,2022-08-28,55.815624,37.714302
3,21908,Восточный,Богородское,"бульвар Маршала Рокоссовского, дом 8, корпус 1",замена лифтового оборудования,2022,1,1,2022-06-23,2022-08-11,55.811977,37.727342
4,21910,Восточный,Богородское,"бульвар Маршала Рокоссовского, дом 8, корпус 3",замена лифтового оборудования,2022,1,1,2022-06-23,2022-08-11,55.811678,37.727592


In [99]:
df.groupby(["unom", "adm_area_name", "district_area_name", "addresse_title", "latitude", "longitude"], as_index=False)["period"].count()

,unom,adm_area_name,district_area_name,addresse_title,latitude,longitude,period
0,365,Восточный,Гольяново,"Алтайская улица, дом 5",55.820525,37.811946,13
1,367,Восточный,Гольяново,"Алтайская улица, дом 7",55.821291,37.813428,12
2,370,Восточный,Гольяново,"Алтайская улица, дом 9, корпус 2",55.821247,37.815237,1
3,376,Восточный,Гольяново,"Алтайская улица, дом 17, корпус 1",55.820776,37.821103,8
4,379,Восточный,Гольяново,"Алтайская улица, дом 19",55.821380,37.822679,1
...,...,...,...,...,...,...,...
368,82032,Восточный,Вешняки,"улица Молдагуловой, дом 16, корпус 3",55.721582,37.816557,4
369,82033,Восточный,Новокосино,"Суздальская улица, дом 6, корпус 2",55.732896,37.849610,7
370,82036,Восточный,Новокосино,"Салтыковская улица, дом 7, корпус 2",55.734776,37.858294,5
371,82037,Восточный,Новокосино,"Суздальская улица, дом 18, корпус 6",55.741458,37.852231,4


In [43]:
# закрытие подключения к базе данных 'postgres'
connect.close()